In [1]:
%run ../../Utils/yp_utils.py

# Initial setup

In [2]:
paper_pmid = 16251356
paper_name = 'reiner_schneiter_2006' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [4]:
datasets.set_index('dataset_id', inplace=True)

# Load & process tested strains

In [11]:
tested = pd.read_excel('raw_data/BY4741-MATa COLLECTION.xls', sheet_name='chr11_1yes')

In [12]:
tested.head()

,acc.-no,ORF,strain,batch,row,col,comments,slow growth?
0,4849.0,YKL001C,BY4741,chr11_1,A,1.0,NaN,NaN
1,4850.0,YKL002W,BY4741,chr11_1,A,2.0,NaN,*
2,4851.0,YKL003C,BY4741,chr11_1,A,3.0,NaN,*
3,4855.0,YKL006W,BY4741,chr11_1,A,7.0,NaN,NaN
4,4856.0,YKL007W,BY4741,chr11_1,A,8.0,NaN,NaN


In [13]:
tested['orf'] = tested['ORF'].astype(str)

In [14]:
tested['orf'] = clean_orf(tested['orf'])

In [15]:
tested['orf'] = translate_sc(tested['orf'], to='orf')

In [16]:
# Make sure everything translated ok
t = looks_like_orf(tested['orf'])
print(tested.loc[~t,])

             acc.-no  ORF strain batch  row  col comments slow growth?  orf
index_input                                                                
1615             NaN  NaN    NaN   NaN  NaN  NaN      NaN          NaN  NAN
3298             NaN  NaN    NaN   NaN  NaN  NaN      NaN          NaN  NAN


In [17]:
tested = tested.loc[t,:]

In [18]:
tested_orfs = tested['orf'].unique()

# Load data

In [19]:
original_data = pd.read_csv('raw_data/reiner_scheiter_2006_hits.txt', sep='\t', header=None)

In [20]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 37 x 1


In [21]:
original_data.head()

,0
0,YGL148
1,YGL084
2,YLR304
3,YLR226
4,YKL139


In [26]:
# The list of hits lacks the last character (was published that way), so we have to match it to the list of tested strains
original_data[1] = ''

for porf in original_data[0].values:
    t = tested.loc[tested['orf'].str.startswith(porf),'orf']
    if t.shape[0] > 0:
        original_data.loc[original_data[0]==porf,1] = t.values[0]
    else:
        print(porf)

YBR039
YNL243


In [30]:
original_data = original_data.loc[original_data[1] != '']

In [31]:
original_data.shape

(35, 2)

In [32]:
original_data['orf'] = original_data[1].astype(str)

In [33]:
original_data['orf'] = clean_orf(original_data['orf'])

In [34]:
original_data['orf'] = translate_sc(original_data['orf'], to='orf')

In [35]:
# Make sure everything translated ok
t = looks_like_orf(original_data['orf'])
print(original_data.loc[~t,])

Empty DataFrame
Columns: [0, 1, orf]
Index: []


In [36]:
original_data['data'] = -1

In [37]:
original_data.set_index('orf', inplace=True)

In [38]:
original_data = original_data[['data']].copy()

In [39]:
original_data = original_data.groupby(original_data.index).mean()

In [40]:
original_data.shape

(35, 1)

In [41]:
original_data = original_data.reindex(index=tested_orfs, fill_value=0)

# Prepare the final dataset

In [42]:
data = original_data.copy()

In [43]:
dataset_ids = [175]
datasets = datasets.reindex(index=dataset_ids)

In [44]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [45]:
data.head()

dataset_id,175
data_type,value
orf,
YKL001C,0
YKL002W,0
YKL003C,0
YKL006W,0
YKL007W,0


## Subset to the genes currently in SGD

In [46]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 22


In [47]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,175
,data_type,value
gene_id,orf,
1399,YKL001C,0
1400,YKL002W,0
1401,YKL003C,0
1404,YKL006W,0
1405,YKL007W,0


# Normalize

In [48]:
data_norm = normalize_phenotypic_scores(data, has_tested=True)

In [49]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [50]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

dataset_id        175       
data_type       value valuez
gene_id orf                 
1399    YKL001C     0    0.0
1400    YKL002W     0    0.0
1401    YKL003C     0    0.0
1404    YKL006W     0    0.0
1405    YKL007W     0    0.0

# Print out

In [51]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [52]:
from IO.save_data_to_db3 import *

In [53]:
save_data_to_db(data_all, paper_pmid)

  0%|          | 0/1 [00:00<?, ?it/s]

Deleting all datasets for PMID 16251356...
Inserting the new data...


100%|██████████| 1/1 [00:08<00:00,  8.67s/it]

Updating the data_modified_on field...
